In [ ]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [ ]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"complete")
    return image_paths,labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

In [ ]:
print(test)

In [ ]:
from tqdm.notebook import tqdm
from PIL import Image


In [ ]:
def extract_features(images, target_size=(48, 48)):
    features = []
    for image in tqdm(images):
        # Load the image in grayscale mode
        img = load_img(image, color_mode='grayscale', target_size=target_size)
        
        # Convert image to array
        img = img_to_array(img)
        
        # Append the image array to features list
        features.append(img)
    
    # Convert features list to a numpy array
    features = np.array(features)
    
    # Ensure features have the correct shape
    features = np.expand_dims(features, axis=-1)  # Adds the channel dimension for grayscale (48, 48, 1)
    
    return features

In [ ]:
train_features = extract_features(train['image']) 

In [ ]:
test_features = extract_features(test['image'])

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
le = LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input

model = Sequential()

# Input layer
model.add(Input(shape=(48, 48, 1)))

# Convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten the output from convolutional layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output layer with 7 classes (for classification)
model.add(Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 


In [18]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [25]:
from keras.models import model_from_json

In [27]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [28]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']


In [29]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [31]:
from keras.utils import load_img, img_to_array
import numpy as np

def ef(image):
    # Load the image in grayscale mode using color_mode='grayscale'
    img = load_img(image, color_mode='grayscale', target_size=(48, 48))
    
    # Convert the image to an array
    feature = img_to_array(img)
    
    # Reshape to add batch dimension (1, 48, 48, 1)
    feature = feature.reshape(1, 48, 48, 1)
    
    # Normalize the pixel values (optional, depending on how your model was trained)
    feature = feature / 255.0
    
    return feature


In [32]:
image = 'images/train/sad/42.jpg'
print("Original image is of sad")

# Preprocess the image
img = ef(image)

# Make a prediction
pred = model.predict(img)

# Get the predicted label
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
pred_label = label[pred.argmax()]

print("Model prediction is", pred_label)


Original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step
Model prediction is sad
